# Extração e tratamento dos dados

## Conexão com o drive e extração das tabelas

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip3 install pyxlsb

In [ ]:
import pyxlsb
import pandas as pd;
import numpy as np;
import seaborn as sns; sns.set();
import matplotlib.pyplot as plt;
import plotly.express as px;
from datetime import datetime

df_compra2022 = pd.read_excel('/content/drive/Shareddrives/grupo4/dados/base2022_compra.xlsb', engine='pyxlsb')
df_compra2023_1 = pd.read_excel('/content/drive/Shareddrives/grupo4/dados/base2023_compra.xlsb', engine='pyxlsb')
df_compra2023_2 = pd.read_excel('/content/drive/Shareddrives/grupo4/dados/base2023_compra_2.xlsb', engine='pyxlsb')
df_venda2022 = pd.read_excel('/content/drive/Shareddrives/grupo4/dados/base2022_venda.xlsb', engine='pyxlsb')
df_venda2023 = pd.read_excel('/content/drive/Shareddrives/grupo4/dados/base2023_venda.xlsb', engine='pyxlsb')


## Renomeação de colunas:

In [ ]:
df_compra2022 = df_compra2022.rename(columns={'Dt. Liquidação': 'Vencimento'})
df_compra2023_1 = df_compra2023_1.rename(columns={'Dt. Liquidação': 'Vencimento'})


## Converter para formato Dia, Mês e Ano

In [ ]:
def converter_data(data):
    try:
        return pd.to_datetime(data, format='%Y-%m-%d')
    except ValueError:
        try:
            return pd.to_datetime(data, format='%d/%m/%Y')
        except ValueError:
            return None

df_compra2022['Dt. Operação'] = df_compra2022['Dt. Operação'].apply(converter_data).dt.date
df_compra2022['Vencimento'] = pd.to_datetime(df_compra2022['Vencimento']).dt.date
df_compra2022['Vencimento'] = pd.to_datetime(df_compra2022['Vencimento'], format='%Y-%d-%m')
df_compra2022['Vencimento'].value_counts()


Vencimento
2022-01-04    565319
Name: count, dtype: int64

In [ ]:
df_compra2023_1['Dt. Operação'] = pd.to_datetime(df_compra2023_1['Dt. Operação'], unit='D', origin='1899-12-30')
df_compra2023_1['Vencimento'] = pd.to_datetime(df_compra2023_1['Vencimento'], unit='D', origin='1899-12-30')

In [ ]:
valores_numericos = df_compra2023_2[df_compra2023_2['Vencimento'].apply(lambda x: isinstance(x, int))]
valores_numericos = df_compra2023_2[df_compra2023_2['Vencimento'].apply(lambda x: str(x).isdigit())]

# Convertendo os valores para inteiros
valores_numericos['Vencimento'] = valores_numericos['Vencimento'].astype(int)


<ipython-input-7-cf4525965571>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valores_numericos['Vencimento'] = valores_numericos['Vencimento'].astype(int)


In [ ]:
df_compra2023_2 = df_compra2023_2.drop(index=valores_numericos.index)
df_compra2023_2['Vencimento'] = pd.to_datetime(df_compra2023_2['Vencimento']).dt.date
valores_numericos['Vencimento'] = pd.to_datetime(valores_numericos['Vencimento'], unit='D', origin='1899-12')

<ipython-input-8-b06d48119250>:2: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_compra2023_2['Vencimento'] = pd.to_datetime(df_compra2023_2['Vencimento']).dt.date


In [ ]:
df_compra2023_2 = pd.concat([df_compra2023_2, valores_numericos])
df_compra2023_2['Dt. Operação'] = df_compra2023_2['Dt. Operação'].apply(converter_data).dt.date
df_compra2023_2['Vencimento'] = pd.to_datetime(df_compra2023_2['Vencimento']).dt.date

## União de tabelas

In [ ]:
df_compra = pd.concat([df_compra2022, df_compra2023_1, df_compra2023_2])

In [ ]:
df_compra.head()

,Cód. Cliente,Dt. Operação,Tipo Operação,Cód. Título,Cód. Corretora,Quantidade,Preço,Valor Líquido,Vencimento,Tipo
0,Cliente 10,2022-04-01,C,ABCB4,STDE,1236,15.560009,-19232.170609,2022-04-01 00:00:00,AV
1,Cliente 10,2022-04-01,C,ABCB4,STDE,1360,15.877823,-21593.839081,2022-04-01 00:00:00,AV
2,Cliente 10,2022-04-01,C,ABCB4,STDE,1755,15.513208,-27225.680443,2022-04-01 00:00:00,AV
3,Cliente 10,2022-04-01,C,ABCB4,STDE,1567,15.668920,-24553.197402,2022-04-01 00:00:00,AV
4,Cliente 10,2022-04-01,C,ABCB4,STDE,1271,15.604854,-19833.769940,2022-04-01 00:00:00,AV


In [ ]:
df_compra['Dt. Operação'] = pd.to_datetime(df_compra['Dt. Operação']).dt.date
df_compra['Vencimento'] = pd.to_datetime(df_compra['Vencimento']).dt.date

In [ ]:
df_venda = pd.concat([df_venda2022, df_venda2023])
df_venda['Dt. Operação'] = pd.to_datetime(df_venda['Dt. Operação'], unit='D', origin='1899-12-30')
df_venda['Vencimento'] = pd.to_datetime(df_venda['Vencimento'], unit='D', origin='1899-12-30')

In [ ]:
df_venda['Dt. Operação'] = pd.to_datetime(df_venda['Dt. Operação']).dt.date
df_venda['Vencimento'] = pd.to_datetime(df_venda['Vencimento']).dt.date

## Verificação de valores nulos

In [ ]:
df_compra.isnull().sum()

Cód. Cliente      0
Dt. Operação      0
Tipo Operação     0
Cód. Título       0
Cód. Corretora    0
Quantidade        0
Preço             0
Valor Líquido     0
Vencimento        0
Tipo              0
dtype: int64

In [ ]:
df_venda.isnull().sum()

Chave             0
Cód. Cliente      0
Dt. Operação      0
Tipo Operação     0
Cód. Título       0
Cód. Corretora    0
Quantidade        0
Preço             0
Valor Líquido     0
Vencimento        0
Tipo              0
DI                0
dtype: int64

## Remoção de Casamentos Determinísticos

Visando evitar vícios no modelo, é necessário retirar os casos em que somente há uma ação a termo para uma combinação da mesma quantidade de ações à vista, sendo esses os casos determinísticos.

In [ ]:
df_ocorrencias_unicas = df_venda.groupby(['Cód. Cliente', 'Dt. Operação', 'Cód. Título', 'Cód. Corretora']).size().reset_index(name='Quantidade de Ocorrências')
df_ocorrencias_unicas_igual_1 = df_ocorrencias_unicas[df_ocorrencias_unicas['Quantidade de Ocorrências'] == 1]
df_ocorrencias_unicas_igual_1 = pd.merge(df_ocorrencias_unicas_igual_1, df_venda[['Cód. Cliente', 'Dt. Operação', 'Cód. Título', 'Cód. Corretora', 'Quantidade']],
                       on=['Cód. Cliente', 'Dt. Operação', 'Cód. Título', 'Cód. Corretora'], how='left')
df_ocorrencias_unicas_igual_1 = df_ocorrencias_unicas_igual_1.drop(columns='Quantidade de Ocorrências')
df_ocorrencias_unicas_igual_1['Dt. Operação'] = pd.to_datetime(df_ocorrencias_unicas_igual_1['Dt. Operação']).dt.date



In [ ]:
df_compra_deterministica = df_compra[['Cód. Cliente', 'Dt. Operação', 'Cód. Título', 'Cód. Corretora', 'Quantidade']]
df_compra_deterministica['Dt. Operação'] = pd.to_datetime(df_compra_deterministica['Dt. Operação']).dt.date
df_compra_deterministica = pd.merge(df_compra_deterministica, df_ocorrencias_unicas_igual_1, on=['Cód. Cliente', 'Dt. Operação', 'Cód. Título', 'Cód. Corretora'], how='inner')


<ipython-input-15-7a2a4dd93369>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_compra_deterministica['Dt. Operação'] = pd.to_datetime(df_compra_deterministica['Dt. Operação']).dt.date


In [ ]:
soma_quantidade_compra = df_compra_deterministica.groupby(['Cód. Cliente', 'Dt. Operação', 'Cód. Título', 'Cód. Corretora', 'Quantidade_y'])['Quantidade_x'].sum().reset_index()
soma_quantidade_compra['Dt. Operação'] = pd.to_datetime(soma_quantidade_compra['Dt. Operação'])

In [ ]:
df_2023 = soma_quantidade_compra[soma_quantidade_compra['Dt. Operação'].dt.year == 2023]
df_2023['Quantidade_x'] = df_2023['Quantidade_x'] - df_2023['Quantidade_y']
soma_quantidade_compra = soma_quantidade_compra[~(soma_quantidade_compra['Dt. Operação'].dt.year == 2023)]
soma_quantidade_compra = pd.concat([soma_quantidade_compra, df_2023], ignore_index=True)

<ipython-input-17-ed08a85c07c4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2023['Quantidade_x'] = df_2023['Quantidade_x'] - df_2023['Quantidade_y']


## Calculo de dias úteis

In [ ]:
import pandas as pd
df_venda['Dt. Operação'] = pd.to_datetime(df_venda['Dt. Operação'])
df_venda['Vencimento'] = pd.to_datetime(df_venda['Vencimento'])

df_venda['dias_uteis'] = df_venda.apply(lambda row: len(pd.bdate_range(row['Dt. Operação'], row['Vencimento'])), axis=1)

df_venda = df_venda[df_venda['dias_uteis'] <= 252]


In [ ]:
df_venda.head()

,Chave,Cód. Cliente,Dt. Operação,Tipo Operação,Cód. Título,Cód. Corretora,Quantidade,Preço,Valor Líquido,Vencimento,Tipo,DI,dias_uteis,preco_ideal_compra
2,B4,Cliente 10,2022-08-26,V,ABCB4,TERR,12907,22.506668,290493.565796,2023-03-22,TE,0.137427,149,24.86
3,B5,Cliente 10,2022-05-30,V,ABEV3,MIRA,5088,14.489812,73724.161850,2022-06-28,TE,0.128568,22,13.70
4,B6,Cliente 10,2022-10-19,V,AERI3,AGOR,226229,1.828252,413603.522337,2023-09-01,TE,0.133300,228,3.05
5,B7,Cliente 10,2022-06-14,V,AERI3,ORAM,113480,3.051308,346262.487914,2022-08-30,TE,0.133586,56,2.47
6,B8,Cliente 10,2022-12-01,V,AERI3,PACT,4404,1.385878,6103.404552,2023-04-06,TE,0.137868,91,1.38


##Calculo do preço ideal de venda

In [ ]:
import pandas as pd

# Suponha que você tenha os dataframes df_venda e df_compra
# 'df_venda' contém as colunas 'dias_uteis' e 'DI'
# 'df_compra' contém a coluna 'Preço'
df_venda_clone = df_venda.copy()

def calculate_a(du, di):
    exp = 252 / du
    a = (di - 1) ** (1 / exp)
    return a

def calculate_pc(pmv, du, di):
    a = calculate_a(du, di)
    return round(abs(pmv / a - 1), 2)  # Trunca para 2 casas decimais

# Adiciona uma nova coluna 'preco_ideal_compra' ao dataframe 'df_compra' com os valores calculados
df_venda_clone['preco_ideal_venda'] = df_venda_clone.apply(lambda row: calculate_pc(row['Preço'], row['dias_uteis'], row['DI']), axis=1)



In [ ]:
df_compra.head()

In [ ]:
df_venda_clone.head()

,Chave,Cód. Cliente,Dt. Operação,Tipo Operação,Cód. Título,Cód. Corretora,Quantidade,Preço,Valor Líquido,Vencimento,Tipo,DI,dias_uteis,preco_ideal_compra,preco_ideal_venda
2,B4,Cliente 10,2022-08-26,V,ABCB4,TERR,12907,22.506668,290493.565796,2023-03-22,TE,0.137427,149,24.86,24.86
3,B5,Cliente 10,2022-05-30,V,ABEV3,MIRA,5088,14.489812,73724.161850,2022-06-28,TE,0.128568,22,13.70,13.70
4,B6,Cliente 10,2022-10-19,V,AERI3,AGOR,226229,1.828252,413603.522337,2023-09-01,TE,0.133300,228,3.05,3.05
5,B7,Cliente 10,2022-06-14,V,AERI3,ORAM,113480,3.051308,346262.487914,2022-08-30,TE,0.133586,56,2.47,2.47
6,B8,Cliente 10,2022-12-01,V,AERI3,PACT,4404,1.385878,6103.404552,2023-04-06,TE,0.137868,91,1.38,1.38


In [ ]:
df_venda_clone.to_excel("df_venda.xlsx", index=False)

In [ ]:
from google.colab import files
files.download("df_venda.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>